# Vector Store Creation with SQL

In [1]:
import teradataml as tdml
import json

## 1 - Connect to Vantage

In [2]:
print('teradataml version    :',tdml.__version__)

teradataml version    : 20.00.00.05


In [ ]:
# Reading JSON data
with open('../connection/connection.json', 'r') as f:
    Param = json.load(f)
    Param['database'] = Param['user']
                
tdml.create_context(**Param)

## 2 - The data

In [6]:
table_name = 'denis_pdfs'
database   = Param['database']

In [5]:
print(tdml.execute_sql(f"SHOW TABLE {database}.{table_name}").fetchall()[0][0].replace('\r','\n'))

CREATE SET TABLE dm250067.denis_pdfs ,FALLBACK ,
     NO BEFORE JOURNAL,
     NO AFTER JOURNAL,
     CHECKSUM = DEFAULT,
     DEFAULT MERGEBLOCKRATIO,
     MAP = TD_MAP2
     (
      TD_ID INTEGER,
      TD_FILENAME VARCHAR(1024) CHARACTER SET LATIN NOT CASESPECIFIC,
      CHUNKS VARCHAR(32000) CHARACTER SET LATIN NOT CASESPECIFIC)
PRIMARY INDEX ( TD_ID );


In [17]:
dataset = tdml.DataFrame(tdml.in_schema(database,table_name))
dataset

select top 10 "TD_ID" AS "TD_ID", "TD_FILENAME" AS "TD_FILENAME", "CHUNKS" AS "CHUNKS" FROM "dm250067"."denis_pdfs";


TD_ID,TD_FILENAME,CHUNKS
80,2404.16130v1.pdf,"alysis. Trade-offs of building a graph index . We consistently observed Graph RAG achieve the best head- to-head results against other methods, but in many cases the graph-free approach to global summa- rization of source texts performed competitively. The real-world decision about whether to invest in building a graph index depends on multiple factors, including the compute budget, expected number of lifetime queries per dataset, and value obtained from other aspects of the graph index (includi"
38,2404.07143v2.pdf,"ss it segment by segment and concatenates back the segments to pass the original-length segment as output to the next layer. Back-propagation through time (BPTT). Each Infini-attention layer is trained with back- propagation through time (Werbos, 1988) by computing the gradient w.r.t the compressive memory states, similar to how RNNs are trained. To save memory, we perform gradient checkpoint when processing the sequence segment by segment. 6Preprint. Under review. Model Memory size (comp.) XL c"
38,2404.16130v1.pdf,"t users may encounter in their real world activities: Podcast transcripts . Compiled transcripts of podcast conversations between Kevin Scott, Microsoft CTO, and other technology leaders (Behind the Tech, Scott, 2024). Size: 1669 600-token text chunks, with 100-token overlaps between chunks ( 1 million tokens). News articles . Benchmark dataset comprising news articles published from September 2013 to December 2023 in a range of categories, including entertainment, business, sports, technology,"
38,Efficiency-Productivity-and-Speed-to-Deployment-MD007074.pdf,"remember that many metrics can be computed directly from other metrics, and so do not need to be explicitly stored. For example, given total spend and total number of transactions, you can compute average spend per transaction. Additional processing that can be done directly against the entity-level EFS does not need to be physically stored. Rather, views can be utilized to give access to the information. Another example would be storing results for each customer for each quarter while"
55,2404.03622v3.pdf,"rends for less powerful models. It suggests their limitations in sustaining reliable reasoning processes across different difficulty levels. Details on performance trends are provided in Appendix D. 6 Related Works Spatial Reasoning over Text Spatial reasoning and spatial language understanding [ KPM20 ] in NLP domain mainly focus on semantic representation [ CBGG97 ,Bat10 ,HK11 ], spatial information extraction [ RMK18 ,KVOM11 ], learning and reasoning [ KM15 ,SLYA17 ,KFP19 ]. Recent advance- m"
55,2404.07143v2.pdf,"ong- context modeling (Bulatov et al., 2022; Chevalier et al., 2023; Ge et al., 2023; Mu et al., 2024; Hwang et al., 2024). However, the previous segment-level compression methods, including Compressive Transformers (Rae et al., 2019) still discard the memory entries of old segments in order to free up space for the new ones, limiting their context window to the most recent segments. This is in contrast to our Infini-attention that computes incremental memory updates to a fixed amount of memory"
118,2404.03622v3.pdf,"g empty squares, as it matches the T shape formed by the empty squares. Let's place Variation 2 of Tetromino T ( ) into the target rectangle: The correct variation of Tetromino T to fit into the empty squares is Variation 2 . Answer: A. 2First, let's look at the empty spaces in the target rectangle. We have a 4x1 space at the bottom and three single squares above it. The Tetromino I ( ) can only fit in the 4x1 space at the bottom, so we'll use Variation 2 of the Tetromino I to fill that"
38,2404.03622v3.pdf,"rtion of instruction se- quences with t=k, i.e., reaching the destination. 4.4 Results As illustrated in Table 1, GPT-4 VoT significantly outperforms other settings in all tasks across all metrics. The significant gap when comparing GPT-4 V oT with GPT-4V CoT and GPT-4

## 3 - Vector Embedding Process : Using BYOM

### 3.1 - Install the model from the Hugging Face page

First we download the model artifacts from the Teradata Hugging Face page: https://huggingface.co/Teradata

In [20]:
from huggingface_hub import hf_hub_download

model_name               = "multilingual-e5-large"
number_dimensions_output = 1024
model_file_name          = "model_int8.onnx"

# Step 1: Download Model from Teradata HuggingFace Page

hf_hub_download(repo_id=f"Teradata/{model_name}", filename=f"onnx/{model_file_name}", local_dir="./")
hf_hub_download(repo_id=f"Teradata/{model_name}", filename=f"tokenizer.json", local_dir="./")


'tokenizer.json'

In [8]:
query = f"""
SELECT
    DatabaseName,
    SUM(CurrentPerm) AS CurrentPermSpace,
    SUM(MaxPerm) AS MaxPermSpace,
    SUM(MaxPerm) - SUM(CurrentPerm) AS AvailablePermSpace    
FROM
    DBC.DiskSpace
WHERE
    DatabaseName = 'dm250067'
GROUP BY
    DatabaseName;
"""
tdml.DataFrame.from_query(query)

DatabaseName,CurrentPermSpace,MaxPermSpace,AvailablePermSpace
dm250067,1169612800,1200000000000,1198830387200


Then we can upload the mode to Vantage.

In [ ]:
tdml.execute_sql("DROP TABLE DM250067.embeddings_models")
tdml.execute_sql("DROP TABLE DM250067.embeddings_tokenizers")

In [ ]:
tdml.save_byom(model_id = model_name, # must be unique in the models table
               model_file = f"onnx/{model_file_name}",
               table_name = 'embeddings_models' )


In [ ]:
# b) Tokenizer
tdml.save_byom(model_id = model_name, # must be unique in the models table
              model_file = 'tokenizer.json',
              table_name = 'embeddings_tokenizers') 


### 3.2 - Creation of the Embedding with ONNXEmbeddings

In [18]:
text_column       = 'CHUNKS'
cols              = [x for x in dataset.columns if x != text_column]
query_input_table = f"""
SELECT 
\t{'\n,\t'.join(cols)}
,\t{text_column} AS txt
FROM {Param['database']}.{table_name}
"""

In [31]:
embeddings_query = f"""
SELECT 
        *
from mldb.ONNXEmbeddings(
        on ({'\n\t'.join(query_input_table.split('\n'))}) as InputTable
        on (select * from embeddings_models where model_id = '{model_name}') as ModelTable DIMENSION
        on (select model as tokenizer from embeddings_tokenizers where model_id = '{model_name}') as TokenizerTable DIMENSION
        using
            Accumulate({','.join(["'"+x+"'" for x in cols + ['txt']])}) 
            ModelOutputTensor('sentence_embedding')
            EnableMemoryCheck('false')
            OutputFormat('FLOAT32(1024))')
            OverwriteCachedModel('false')
    ) a 
"""
print(embeddings_query)


SELECT 
        *
from mldb.ONNXEmbeddings(
        on (
	SELECT 
		TD_ID
	,	TD_FILENAME
	,	CHUNKS AS txt
	FROM dm250067.denis_pdfs
	) as InputTable
        on (select * from embeddings_models where model_id = 'multilingual-e5-large') as ModelTable DIMENSION
        on (select model as tokenizer from embeddings_tokenizers where model_id = 'multilingual-e5-large') as TokenizerTable DIMENSION
        using
            Accumulate('TD_ID','TD_FILENAME','txt') 
            ModelOutputTensor('sentence_embedding')
            EnableMemoryCheck('false')
            OutputFormat('FLOAT32(1024))')
            OverwriteCachedModel('false')
    ) a 



In [32]:
DF_embeddings = tdml.DataFrame.from_query(embeddings_query)
DF_embeddings

select top 10 "TD_ID" AS "TD_ID", "TD_FILENAME" AS "TD_FILENAME", txt AS txt, emb_0 AS emb_0, emb_1 AS emb_1, emb_2 AS emb_2, emb_3 AS emb_3, emb_4 AS emb_4, emb_5 AS emb_5, emb_6 AS emb_6, emb_7 AS emb_7, emb_8 AS emb_8, emb_9 AS emb_9, emb_10 AS emb_10, emb_11 AS emb_11, emb_12 AS emb_12, emb_13 AS emb_13, emb_14 AS emb_14, emb_15 AS emb_15, emb_16 AS emb_16, emb_17 AS emb_17, emb_18 AS emb_18, emb_19 AS emb_19, emb_20 AS emb_20, emb_21 AS emb_21, emb_22 AS emb_22, emb_23 AS emb_23, emb_24 AS emb_24, emb_25 AS emb_25, emb_26 AS emb_26, emb_27 AS emb_27, emb_28 AS emb_28, emb_29 AS emb_29, emb_30 AS emb_30, emb_31 AS emb_31, emb_32 AS emb_32, emb_33 AS emb_33, emb_34 AS emb_34, emb_35 AS emb_35, emb_36 AS emb_36, emb_37 AS emb_37, emb_38 AS emb_38, emb_39 AS emb_39, emb_40 AS emb_40, emb_41 AS emb_41, emb_42 AS emb_42, emb_43 AS emb_43, emb_44 AS emb_44, emb_45 AS emb_45, emb_46 AS emb_46, emb_47 AS emb_47, emb_48 AS emb_48, emb_49 AS emb_49, emb_50 AS emb_50, emb_51 AS emb_51, emb_52

TD_ID       TD_FILENAME                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                txt     emb_0     emb_1     emb_2     emb_3     emb_4     emb_5     emb_6     emb_7     emb_8     emb_9    emb_10    emb_11    emb_12    emb_13    emb_14    emb_15    emb_16    emb_17    emb_18    emb_19    emb_20    emb_21    emb_22    emb_23    emb_24    emb_25    emb_26    emb_27    emb_28    emb_29    emb_30    emb_31    emb_32    emb_33    emb_34    emb_35    emb_36    emb_37    emb_38    emb_39    emb_40    emb_41    emb_42    emb_43    emb_44    emb_45    emb_46    emb_47    emb_48    emb_49    emb_50    emb_51    emb_52    emb_53    emb_54    emb_55    emb_56    emb_57    emb_58    emb_59    emb_60    emb_61    emb_62    emb_63    emb_64    emb_65    emb_66    emb_67    emb_68    emb_69    emb_70    emb_71    emb_72    emb_73    emb_74    emb_75    emb_76    emb_77    emb_78    emb_79    emb_80    emb_81    emb_82    emb_83    emb_84    emb_85    emb_86    emb_87    emb_88    emb_89    emb_90    emb_91    emb_92    emb_93    emb_94    emb_95    emb_96    emb_97    emb_98    emb_99   emb_100   emb_101   emb_102   emb_103   emb_104   emb_105   emb_106   emb_107   emb_108   emb_109   emb_110   emb_111   emb_112   emb_113   emb_114   emb_115   emb_116   emb_117   emb_118   emb_119   emb_120   emb_121   emb_122   emb_123   emb_124   emb_125   emb_126   emb_127   emb_128   emb_129   emb_130   emb_131   emb_132   emb_133   emb_134   emb_135   emb_136   emb_137   emb_138   emb_139   emb_140   emb_141   emb_142   emb_143   emb_144   emb_145   emb_146   emb_147   emb_148   emb_149   emb_150   emb_151   emb_152   emb_153   emb_154   emb_155   emb_156   emb_157   emb_158   emb_159   emb_160   emb_161   emb_162   emb_163   emb_164   emb_165   emb_166   emb_167   emb_168   emb_169   emb_170   emb_171   emb_172   emb_173   emb_174   emb_175   emb_176   emb_177   emb_178   emb_179   emb_180   emb_181   emb_182   emb_183   emb_184   emb_185   emb_186   emb_187   emb_188   emb_189   emb_190   emb_191   emb_192   emb_193   emb_194   emb_195   emb_196   emb_197   emb_198   emb_199   emb_200   emb_201   emb_202   emb_203   emb_204   emb_205   emb_206   emb_207   emb_208   emb_209   emb_210   emb_211   emb_212   emb_213   emb_214   emb_215   emb_216   emb_217   emb_218   emb_219   emb_220   emb_221   emb_222   emb_223   emb_224   emb_225   emb_226   emb_227   emb_228   emb_229   emb_230   emb_231   emb_232   emb_233   emb_234   emb_235   emb_236   emb_237   emb_238   emb_239   emb_240   emb_241   emb_242   emb_243   emb_244   emb_245   emb_246   emb_247   emb_248   emb_249   emb_250   emb_251   emb_252   emb_253   emb_254   emb_255   emb_256   emb_257   emb_258   emb_259   emb_260   emb_261   emb_262   emb_263   emb_264   emb_265   emb_266   emb_267   emb_268   emb_269   emb_270   emb_271   emb_272   emb_273   emb_274   emb_275   emb_276   emb_277   emb_278   emb_279   emb_280   emb_281   emb_282   emb_283   emb_284   emb_285   emb_286   emb_287   emb_288   emb_289   emb_290   emb_291   emb_292   emb_293   emb_294   emb_295   emb_296   emb_297   emb_298   emb_299   emb_300   emb_301   emb_302   emb_303   emb_304   emb_305   emb_306   emb_307   emb_308   emb_309   emb_310   emb_311   emb_312   emb_313   emb_314   emb_315   emb_316   emb_317   emb_318   emb_319   emb_320   emb_321   emb_322   emb_323   emb_324   emb_325   emb_326   emb_327   emb_328   emb_329   emb_330   emb_331   emb_332   emb_333   emb_334   emb_335   emb_336   emb_337   emb_338   emb_339   emb_340   emb_341   emb_342   emb_343   emb_344   emb_345  

In [23]:
vector_store_table   = 'denis_pdfs_embeddings'
primary_index        = ['TD_ID']

In [24]:
tdml.execute_sql(f"DROP TABLE {database}.{vector_store_table}")

TeradataCursor uRowsHandle=74 bClosed=False

In [25]:
embedding_query = f"""
CREATE MULTISET TABLE {database}.{vector_store_table} AS
(
SELECT 
        {','.join([x for x in cols + ['txt']])}
        , CAST(sentence_embedding AS VECTOR) AS sentence_embedding
from mldb.ONNXEmbeddings(
        on ({'\n\t'.join(query_input_table.split('\n'))}) as InputTable
        on (select * from embeddings_models where model_id = '{model_name}') as ModelTable DIMENSION
        on (select model as tokenizer from embeddings_tokenizers where model_id = '{model_name}') as TokenizerTable DIMENSION
        using
            Accumulate({','.join(["'"+x+"'" for x in cols + ['txt']])}) 
            ModelOutputTensor('sentence_embedding')
            EnableMemoryCheck('false')
            OutputFormat('varbyte({number_dimensions_output*4*2})')
            OverwriteCachedModel('false')
    ) a 
) WITH DATA
PRIMARY INDEX ({','.join(primary_index)})
"""

print(embedding_query)


CREATE MULTISET TABLE dm250067.denis_pdfs_embeddings AS
(
SELECT 
        TD_ID,TD_FILENAME,txt
        , CAST(sentence_embedding AS VECTOR) AS sentence_embedding
from mldb.ONNXEmbeddings(
        on (
	SELECT 
		TD_ID
	,	TD_FILENAME
	,	CHUNKS AS txt
	FROM dm250067.denis_pdfs
	) as InputTable
        on (select * from embeddings_models where model_id = 'multilingual-e5-large') as ModelTable DIMENSION
        on (select model as tokenizer from embeddings_tokenizers where model_id = 'multilingual-e5-large') as TokenizerTable DIMENSION
        using
            Accumulate('TD_ID','TD_FILENAME','txt') 
            ModelOutputTensor('sentence_embedding')
            EnableMemoryCheck('false')
            OutputFormat('varbyte(8192)')
            OverwriteCachedModel('false')
    ) a 
) WITH DATA
PRIMARY INDEX (TD_ID)



In [26]:
%%time
tdml.execute_sql(embedding_query)

CPU times: total: 141 ms
Wall time: 3min 57s


TeradataCursor uRowsHandle=75 bClosed=False

In [33]:
vector_table = tdml.DataFrame(tdml.in_schema(database, vector_store_table))
vector_table

select top 10 "TD_ID" AS "TD_ID", "TD_FILENAME" AS "TD_FILENAME", txt AS txt, CAST(sentence_embedding AS VARCHAR(1024)) AS sentence_embedding FROM "dm250067"."denis_pdfs_embeddings";


TD_ID,TD_FILENAME,txt,sentence_embedding
80,2404.07143v2.pdf,"tan Belinkov, Ori Ram, Omri Abend, Ehud Karpas, Amnon Shashua, Kevin Leyton-Brown, and Yoav Shoham. Parallel context windows improve in-context learning of large language models. arXiv preprint arXiv:2212.10947 , 2022. Imanol Schlag, Paul Smolensky, Roland Fernandez, Nebojsa Jojic, J urgen Schmidhuber, and Jianfeng Gao. Enhancing the transformer with explicit relational encoding for math problem solving. arXiv preprint arXiv:1910.06611 , 2019. 12Preprint. Under review. Imanol Schlag, Tsendsuren","6.43728e-16,-4.41093e-17,3.15184e-25,1.59152e-12,-1.3639e-25,4.65858e-21,1.23374e-26,-4.70707e-19,1.30649e-14,-7.65718e-14,3.15724e-16,-8.7788e-16,3.56691e-19,-5.41717e-15,-1.87855e-15,-1.85183e-24,-9.85414e-14,3.26196e-21,1.09714e-13,4.48756e-16,5.66333e-14,-2.90041e-16,4.06488e-15,-4.30782e-19,1.33064e-15,-1.48521e-20,-9.01422e-15,1.23537e-21,6.28377e-16,-3.60983e-18,-9.54965e-18,-2.98719e-15,1.75305e-15,-8.87107e-17,1.11562e-21,4.9385e-14,-2.29381e-14,1.6397e-28,-8.00761e-20,1.26816e-18,-1.51094e-17,-1.0787e-13,1.6034e-13,7.6601e-14,5.76154e-17,7.20712e-13,-2.00635e-19,1.28397e-18,5.21045e-16,-1.17864e-17,1.75788e-19,-2.16055e-14,4.65835e-17,1.27485e-13,-2.21663e-17,3.15685e-13,-2.67249e-16,-5.8853e-19,-6.88681e-19,1.08851e-15,1.17169e-15,-1.97453e-13,-1.49793e-26,3.27029e-13,3.69905e-16,4.39526e-16,-6.39523e-20,-8.06223e-14,-3.25322e-15,5.41515e-19,2.73785e-16,-5.43152e-14,-9.58957e-14,3.15205e-24,-1.75985e-32,-9.60334e-14,8.55388e-14,-1.19865e-13,5.31283e-12,2.22846e-15,-2.02938e-14,-5.59565e-15,-3.3449e"
38,2404.16130v1.pdf,"t users may encounter in their real world activities: Podcast transcripts . Compiled transcripts of podcast conversations between Kevin Scott, Microsoft CTO, and other technology leaders (Behind the Tech, Scott, 2024). Size: 1669 600-token text chunks, with 100-token overlaps between chunks ( 1 million tokens). News articles . Benchmark dataset comprising news articles published from September 2013 to December 2023 in a range of categories, including entertainment, business, sports, technology,","1.05358e-17,-7.15347e-16,-3.61777e-15,2.49678e-12,-3.57585e-18,2.2217e-20,-6.84316e-18,-2.81663e-19,5.7331e-17,-9.60244e-15,-7.08555e-16,-4.51617e-18,-3.78458e-16,-1.71315e-14,-4.31512e-15,2.91654e-19,-7.94995e-16,2.71884e-14,-2.48338e-30,-1.37004e-16,2.40251e-14,-1.3702e-14,4.10585e-16,-2.01766e-15,6.66141e-14,-1.12517e-23,-4.44021e-17,2.2335e-13,6.94763e-14,2.74884e-17,-9.70385e-21,-1.22428e-17,6.52126e-13,1.98213e-26,4.88781e-20,4.62922e-17,-1.55553e-14,-1.01661e-18,-5.74501e-18,-2.95758e-21,-4.06136e-17,-2.27358e-12,1.28156e-14,3.10988e-13,1.71918e-14,9.09548e-14,-2.79748e-17,6.88324e-24,-4.31453e-20,-7.49873e-16,-4.31351e-16,-7.77434e-15,2.4713e-16,2.06949e-14,-1.60996e-18,8.50447e-12,-2.66929e-16,1.22514e-17,-5.22196e-23,5.99009e-18,1.38667e-15,-3.23221e-15,-2.40461e-24,1.967e-12,3.4567e-17,3.2674e-18,9.55266e-19,-2.0382e-12,-5.06777e-15,-1.00253e-19,1.33431e-17,-7.00774e-14,-6.09005e-14,-1.58655e-25,-2.68149e-15,-2.76317e-15,2.23126e-15,-1.05774e-14,9.15341e-14,1.65433e-19,-5.08633e-14,-2.31969e-18,-3."
38,2404.07143v2.pdf,"ss it segment by segment and concatenates back the segments to pass the original-length segment as output to the next layer. Back-propagation through time (BPTT). Each Infini-attention layer is trained with back- propagation through time (Werbos, 1988) by computing the gradient w.r.t the compressive memory states, similar to how RNNs are trained. To save memory, we perform gradient checkpoint when processing the sequence segment by segment. 6Preprint. Under review. Model Memory size (comp.) XL c","-3.06824e-21,-2.22778e-14,-3.01441e-14,5.10925e-12,-7.73342e-15,1.42968e-15,1.16465e-18,-7.30718e-19,9.65143e-15,-4.38958e-14,-1.15163e-18,-1.97983e-14,5.37961e-21,-2.26251e-17,-8.76396e-15,1.25558e-20,-4.49754e-16,6.59547e-15,5.07394e-16,1.52303e-20,3.37016e-15,-6.75224e-17,2.03828e-17,-4.93261e-16,-3.86089e-22,-7.30962e-19,-3.98335e-

In [ ]:
vector_table = tdml.DataFrame(tdml.in_schema(database, vector_store_table))
df = vector_table.assign(vector = vector_table.sentence_embedding.cast(tdml.VARCHAR(length=32000))).to_pandas(num_rows=5)
df.vector.values[0] # the string of a comma separated vector
len(df.vector.values[0].split(',')) # this should return 1024 but gives 512
df

,TD_FILENAME,txt,sentence_embedding,vector
TD_ID,,,,
145,2404.03622v3.pdf,"al\nimage correctly.\nStarting from \n , let's...","b'\x06\xabU<;\xb1\xef\xbb\xbc\t;;k""\xfc\xbc3+\...","-5.36889e-20,-6.2471e-15,-1.24765e-15,8.32693e..."
40,2404.16130v1.pdf,"which people inspect, engage with, and context...",b'_\x96\xfd<v\xac\x01<\x9e\x86\xb7\xbc\xfa\x1e...,"1.19762e-19,-5.30715e-16,-1.15475e-14,4.93502e..."
55,Efficiency-Productivity-and-Speed-to-Deploymen...,o-the-second \ndata should ever be used and th...,b'e\x15\xdb<\xecQ=\xbc\xa7%\x86\xbc3b\xa9\xbc\...,"-1.58944e-18,-1.76134e-16,-8.79388e-17,4.28712..."
99,2404.03622v3.pdf,"D coordinate system with origin (0, 0) as the ...","b'+\xa94<j!\x06\xbc\xb8m\xd1\xbc\xcbz\x9a\xbc,...","-1.49962e-19,-9.18698e-17,-1.48638e-15,1.74875..."
42,Efficiency-Productivity-and-Speed-to-Deploymen...,"ssing time, more complex scripts, and more sto...",b'\x99\x0f\x1c<~\xc9\x00;{9\xca\xbc\xb1\x07\x9...,"1.73574e-24,-7.64321e-17,-8.12929e-17,1.0877e-..."


In [1]:
import teradataml as tdml


In [ ]:
vector_table = tdml.DataFrame(tdml.in_schema(database, vector_store_table))
df = vector_table.assign(vector = vector_table.sentence_embedding.cast(tdml.VARCHAR(length=32000)))
df

select top 10 "TD_ID" AS "TD_ID", "TD_FILENAME" AS "TD_FILENAME", txt AS txt, CAST(sentence_embedding AS VARCHAR(1024)) AS sentence_embedding, vector AS vector FROM ( select "TD_ID" AS "TD_ID", "TD_FILENAME" AS "TD_FILENAME", txt AS txt, sentence_embedding AS sentence_embedding, CAST(sentence_embedding AS VARCHAR(32000)) AS vector from "dm250067"."denis_pdfs_embeddings" ) as temp_table;


TD_FILENAME                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 txt                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                sentence_embedding                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               

In [37]:
df.vector.values[0]

'-5.36889e-20,-6.2471e-15,-1.24765e-15,8.32693e-12,-1.97777e-16,2.31327e-19,-2.50576e-23,-1.76153e-17,2.27939e-14,-4.48308e-14,-1.31066e-16,-6.32158e-14,-1.39564e-19,-2.15604e-18,-1.04195e-13,1.34092e-20,-1.02485e-19,1.43682e-17,5.55539e-17,3.33934e-15,5.10329e-16,-4.70506e-14,6.69692e-21,2.15765e-20,2.0467e-16,2.45468e-14,-1.19508e-19,4.84034e-15,2.1264e-20,-5.32071e-21,-3.62742e-17,-8.61437e-17,-4.7975e-27,3.19244e-18,1.46614e-15,6.91292e-17,-3.25213e-15,-7.12507e-17,-9.07408e-17,-7.46461e-21,-6.82539e-22,-2.48204e-20,8.76461e-16,2.58427e-15,1.72067e-14,4.71732e-14,-7.89991e-19,3.43869e-22,-2.16597e-19,4.06285e-21,-1.04188e-21,-9.20345e-16,2.0408e-16,2.87422e-16,-2.63033e-21,6.6078e-13,3.23867e-24,8.47182e-19,-3.65446e-16,1.07815e-17,2.48621e-17,-3.69116e-13,-2.14535e-17,2.61891e-14,2.11704e-16,5.16786e-16,-1.83606e-26,-9.07321e-14,3.67721e-24,-2.91805e-17,6.40195e-15,-6.39699e-15,-9.86223e-14,2.7771e-16,-3.98536e-17,-2.0051e-16,1.51175e-17,-2.60665e-22,2.3203e-14,6.41113e-17,-3.4994

In [2]:
txt = '-5.36889e-20,-6.2471e-15,-1.24765e-15,8.32693e-12,-1.97777e-16,2.31327e-19,-2.50576e-23,-1.76153e-17,2.27939e-14,-4.48308e-14,-1.31066e-16,-6.32158e-14,-1.39564e-19,-2.15604e-18,-1.04195e-13,1.34092e-20,-1.02485e-19,1.43682e-17,5.55539e-17,3.33934e-15,5.10329e-16,-4.70506e-14,6.69692e-21,2.15765e-20,2.0467e-16,2.45468e-14,-1.19508e-19,4.84034e-15,2.1264e-20,-5.32071e-21,-3.62742e-17,-8.61437e-17,-4.7975e-27,3.19244e-18,1.46614e-15,6.91292e-17,-3.25213e-15,-7.12507e-17,-9.07408e-17,-7.46461e-21,-6.82539e-22,-2.48204e-20,8.76461e-16,2.58427e-15,1.72067e-14,4.71732e-14,-7.89991e-19,3.43869e-22,-2.16597e-19,4.06285e-21,-1.04188e-21,-9.20345e-16,2.0408e-16,2.87422e-16,-2.63033e-21,6.6078e-13,3.23867e-24,8.47182e-19,-3.65446e-16,1.07815e-17,2.48621e-17,-3.69116e-13,-2.14535e-17,2.61891e-14,2.11704e-16,5.16786e-16,-1.83606e-26,-9.07321e-14,3.67721e-24,-2.91805e-17,6.40195e-15,-6.39699e-15,-9.86223e-14,2.7771e-16,-3.98536e-17,-2.0051e-16,1.51175e-17,-2.60665e-22,2.3203e-14,6.41113e-17,-3.49948e-14,-8.95051e-19,-9.22649e-15,3.51766e-17,-9.9127e-15,-8.48035e-13,-3.29136e-14,1.37333e-15,4.41526e-19,-1.03766e-15,4.83694e-16,-1.20585e-14,2.43524e-17,2.99156e-15,4.79504e-14,-7.41907e-19,-2.90626e-21,1.45483e-15,4.31243e-18,-7.64628e-14,5.25213e-16,5.47288e-17,-4.37945e-18,-1.10783e-13,-1.82041e-13,-1.03289e-16,2.38934e-18,-2.59953e-12,-3.34165e-16,-4.2987e-17,4.05281e-13,7.78651e-33,4.63837e-14,2.75041e-14,-1.17836e-19,2.33412e-17,-9.64009e-17,4.69134e-23,-1.62132e-15,3.7522e-17,-2.81054e-14,-6.66878e-16,-3.44843e-15,-8.12214e-16,-1.84972e-16,1.65639e-17,1.57729e-18,7.78142e-14,-4.42489e-17,-1.87843e-18,-1.57067e-27,1.37204e-17,1.13714e-19,1.66032e-34,-2.06939e-15,-1.48026e-14,-4.16929e-14,-2.3682e-21,-6.36042e-17,2.39649e-23,-3.68979e-16,-3.21085e-13,-2.05884e-14,-2.34008e-23,-3.57933e-23,3.83254e-23,2.00702e-15,1.4793e-14,-2.45235e-10,4.04146e-17,-2.3547e-16,1.02385e-19,-1.79854e-18,5.6069e-17,5.49239e-23,1.75763e-14,-1.98067e-15,-1.81519e-21,5.68146e-24,3.23443e-17,-3.92729e-18,2.47572e-15,-1.13321e-19,-7.87242e-17,-2.26803e-15,-6.395e-15,1.3471e-12,1.27591e-14,2.81503e-17,-6.44609e-27,-2.93357e-23,-5.69967e-19,-8.16392e-15,-2.02488e-16,1.53576e-15,-3.71537e-14,-6.88324e-24,2.21443e-12,-2.49743e-14,-3.18078e-16,4.42213e-17,-3.97688e-16,-1.9964e-19,-2.37928e-19,-1.82747e-16,4.64481e-16,3.8082e-18,-3.98022e-20,-3.64546e-20,-1.75419e-15,-1.7838e-13,-8.04796e-14,-9.63281e-14,2.44797e-21,-3.34188e-12,9.84912e-19,1.31873e-18,3.43922e-17,-8.14273e-20,-7.0169e-15,1.25486e-14,5.00768e-14,-4.76807e-20,-6.18028e-14,5.65404e-14,-1.12886e-16,1.78553e-16,2.4966e-18,2.31276e-17,-3.82206e-16,6.30865e-19,-7.42882e-23,-3.34858e-18,-7.16441e-20,4.54213e-14,3.25654e-18,-1.79647e-16,-5.96156e-24,6.35501e-15,-1.82489e-16,-1.33691e-14,-1.9843e-19,6.46628e-16,1.01968e-16,-1.91339e-13,3.75076e-15,-3.20664e-15,-7.18966e-15,-1.0915e-13,-1.27101e-18,-4.99977e-19,2.87884e-14,-4.44397e-21,-1.74931e-16,8.12769e-20,-9.8917e-17,1.21678e-25,6.93698e-15,2.3659e-15,3.85349e-12,-2.41657e-13,3.31057e-20,-5.60797e-13,2.1162e-29,-1.36744e-14,-1.40581e-14,2.9801e-14,-4.56837e-16,4.78283e-19,-2.06625e-19,1.7752e-12,-8.65056e-16,2.08151e-20,2.23647e-14,2.37022e-14,3.09149e-13,-1.12712e-18,4.55718e-21,-7.36359e-15,2.04941e-19,-3.293e-19,-2.38469e-16,3.92755e-17,1.46783e-14,7.36961e-16,-5.0752e-22,-6.88197e-12,-3.85218e-14,-1.11211e-16,4.53406e-20,-3.37035e-13,1.26222e-16,-7.98364e-16,-3.10329e-18,1.10174e-13,1.77802e-13,5.89336e-14,5.47242e-14,-1.28175e-14,7.43243e-15,-4.34936e-16,1.37142e-14,3.3117e-21,1.90116e-17,-1.90944e-18,1.93413e-24,-3.65886e-15,1.28142e-15,1.54661e-12,1.35562e-19,-2.00596e-19,1.95277e-16,-6.33282e-15,-1.27467e-14,5.15605e-26,-8.07267e-15,1.3001e-17,-1.32965e-32,-3.33362e-16,-4.25164e-22,7.84937e-16,4.90816e-14,-1.18868e-16,6.78676e-16,-3.79388e-14,-9.35269e-29,9.07651e-15,-1.63062e-19,-7.34638e-25,-3.76867e-15,-1.54559e-15,5.66464e-22,-1.21966e-15,-4.46288e-16,-5.5729e-21,9.32153e-14,-4.69124e-17,2.17101e-18,-5.37583e-16,6.85753e-12,-9.38233e-14,-1.37563e-20,-4.99396e-15,1.05816e-14,-4.83938e-18,5.47781e-17,1.14374e-22,-1.61209e-12,-4.74205e-16,-5.19962e-12,3.29652e-12,-1.18663e-14,-6.36562e-15,1.13226e-14,-2.37122e-20,1.03271e-14,1.5819e-13,-9.50446e-15,4.02978e-14,3.65638e-17,-1.58007e-13,-3.48029e-13,-6.2805e-15,-1.04457e-13,-1.07363e-14,3.83489e-19,8.86962e-19,5.62262e-14,-4.66234e-13,2.19568e-13,-4.9077e-22,-1.06856e-23,1.482e-12,-5.28351e-17,-3.2923e-17,-7.08125e-17,5.45901e-16,4.4353e-18,6.17381e-26,3.8937e-17,-7.2371e-14,-6.83398e-14,-4.71728e-14,6.72665e-14,-1.21229e-14,-1.11712e-16,1.14522e-15,-9.02618e-17,1.63546e-13,4.67161e-20,1.16786e-14,2.53416e-16,-2.39483e-20,2.29531e-20,3.82594e-23,2.43265e-15,1.23003e-18,6.15541e-17,-2.96427e-22,-8.72909e-15,1.01958e-14,1.44767e-18,-2.80196e-21,-3.43622e-14,-4.51537e-17,-1.44206e-14,2.51379e-18,7.1566e-15,4.96892e-14,2.44868e-19,1.61351e-14,-3.42475e-16,-1.82592e-18,5.47397e-14,-3.63939e-17,4.51734e-18,-9.63912e-21,1.51452e-14,-1.50579e-15,7.85423e-17,-5.00356e-14,4.27272e-19,2.12264e-16,3.67484e-14,-3.0454e-13,-3.95494e-15,4.85957e-14,1.72024e-15,-1.8293e-19,6.19574e-16,4.33221e-19,4.71995e-22,1.94751e-14,-1.6772e-18,-5.19648e-16,-1.7167e-16,1.69301e-13,2.7173e-12,-1.6605e-18,4.85471e-14,-4.17247e-14,-2.83618e-14,-1.84179e-19,2.55214e-18,-3.22624e-21,-1.04015e-15,4.59864e-17,-8.73219e-16,6.0736e-16,2.49464e-18,-8.35457e-17,3.53289e-16,2.27113e-16,-6.78611e-18,5.56381e-15,2.49599e-12,-4.28159e-14,-1.62881e-19,-1.33916e-14,1.45798e-20,-1.05833e-17,-5.3032e-18,9.59367e-17,-9.28744e-19,-2.04026e-12,-3.81955e-15,-6.23272e-15,1.19393e-19,2.70094e-36,4.42915e-14,-9.07443e-13,-3.86396e-17,-4.42604e-17,-8.46674e-14,-1.80614e-15,-2.1283e-16,2.77443e-21,2.75214e-28,3.90182e-11,4.80332e-14,-1.84577e-14,-2.61959e-14,4.754e-20,-4.60379e-14,-1.99128e-16,-2.48804e-16,6.68663e-14,8.39416e-17,-3.1047e-16,1.67833e-17,-1.76649e-23,4.90943e-12,-9.71346e-15,6.6856e-17,-5.07193e-21,1.67935e-14,1.97474e-15,-1.23439e-16,5.13297e-17,6.93136e-17,6.02029e-24,1.99321e-15,1.59386e-12,2.46909e-14,3.0972e-15,3.28554e-15,1.05158e-14,-4.07577e-15,6.69682e-21,-8.39534e-21,9.30716e-26,-3.90376e-16,-2.55284e-15,8.30882e-18,3.95429e-15,-1.25513e-16,5.20919e-13,1.12784e-16,-3.97842e-13,2.56317e-17,1.99073e-14,-7.51405e-24,7.64393e-17,4.77604e-15,-2.77884e-14,2.12029e-15,-3.92704e-19,-1.02841e-14,-2.07688e-16,1.4927e-13,2.30909e-32,1.66479e-19'
len(txt)

6347

In [38]:
len(df.vector.values[0].split(','))

512

In [49]:
len(df.vector.values[0])

c:\Users\dm250067\OneDrive - Teradata\Documents\01 - Code Development\demo-vector-store\.venv\Lib\site-packages\teradataml\dataframe\sql.py:10717: SyntaxWarning: invalid escape sequence '\d'
  func.regexp_replace(func.regexp_substr(self.expression, '//([^/?#]*)'), '(//[^/?#]+@)|(//)|(:\d+)', ''),
c:\Users\dm250067\OneDrive - Teradata\Documents\01 - Code Development\demo-vector-store\.venv\Lib\site-packages\teradataml\dataframe\sql.py:10724: SyntaxWarning: invalid escape sequence '\?'
  '\?([^#]*)'), '?'), type=VARCHAR()),
c:\Users\dm250067\OneDrive - Teradata\Documents\01 - Code Development\demo-vector-store\.venv\Lib\site-packages\teradataml\dataframe\sql.py:10727: SyntaxWarning: invalid escape sequence '\?'
  '^(([^:/?#]+):)?(//([^/?#]*))?([^?#]*)(\?([^#]*))?', ''),
c:\Users\dm250067\OneDrive - Teradata\Documents\01 - Code Development\demo-vector-store\.venv\Lib\site-packages\teradataml\dataframe\sql.py:10734: SyntaxWarning: invalid escape sequence '\?'
  '([^?#]*)(\?([^#]*))?'), typ

AttributeError: '_SQLColumnExpression' object has no attribute 'values'

In [42]:
len('-1.08668e-23,-8.38863e-17,-1.68725e-14,6.09481e-12,-2.61027e-17,5.49815e-15,-9.11604e-24,-5.71803e-20,6.2054e-15,-1.06939e-15,-1.56427e-14,-1.61474e-15,-4.26515e-14,-3.18455e-20,-9.61656e-18,3.81575e-17,-3.15913e-16,1.23304e-16,5.51392e-14,2.6534e-14,2.11093e-14,-2.86068e-14,-2.06027e-16,-2.17444e-15,-1.97737e-21,1.12713e-17,-4.45558e-17,2.20851e-19,-1.58396e-22,2.82563e-16,-1.57765e-16,-1.76483e-15,5.13024e-17,-3.95336e-18,-3.14162e-29,1.25582e-14,-5.17659e-17,-1.77439e-18,-1.76176e-16,-3.33907e-24,-9.90975e-15,-1.08691e-14,2.49743e-15,8.17084e-14,4.94833e-17,1.10987e-13,-2.60064e-18,1.00588e-16,-1.33177e-17,8.24316e-24,2.60431e-19,-1.87382e-13,1.81284e-23,4.59156e-20,5.7229e-21,1.73087e-16,-1.9062e-16,-4.74507e-19,-7.74557e-16,4.03376e-25,2.90085e-17,-5.42029e-13,-8.60066e-17,4.79003e-14,7.8699e-17,1.19824e-14,2.64726e-33,-3.04495e-14,6.27227e-22,3.71611e-18,5.56041e-14,-2.07708e-13,-5.37779e-15,1.51282e-22,-3.13351e-17,-5.07865e-18,5.28074e-27,-6.1907e-14,3.55174e-14,5.04606e-15,-8.10712e-13,-4.90654e-17,-4.42282e-14,6.03208e-17,-5.86198e-16,-4.91869e-13,-8.47756e-12,5.05601e-15,8.32781e-17,-1.61176e-14,6.11114e-17,4.19634e-17,7.29077e-18,5.26367e-17,3.56421e-16,3.77776e-22,1.48139e-16,5.29807e-14,-2.6544e-21,-3.4521e-15,5.16427e-16,1.63959e-23,-2.98885e-20,-2.55533e-16,-5.1067e-14,-5.04867e-16,1.5417e-22,-1.79615e-12,-3.52633e-17,-5.46237e-16,6.73562e-13,7.17307e-17,5.86539e-16,3.75626e-13,1.56769e-16,8.60966e-16,-7.70546e-14,-3.1134e-18,1.5134e-19,-6.79793e-26,-2.02269e-20,-2.33876e-22,-3.21231e-19,-2.69463e-15,-1.52162e-14,1.71232e-14,-3.16799e-28,-9.37616e-20,-1.59132e-16,-1.21923e-18,2.04483e-19,1.89772e-16,2.17358e-16,-1.2823e-19,-3.28201e-12,-2.16663e-14,-1.36232e-14,8.42443e-21,-8.33071e-17,1.50272e-16,-1.34193e-14,-6.97211e-13,-6.02458e-21,-1.76116e-19,1.29205e-18,-1.68466e-27,1.82134e-14,-2.65382e-16,-4.32351e-10,-1.57436e-22,-4.00512e-15,6.24643e-22,-5.1352e-16,2.82359e-16,1.47334e-17,1.15443e-15,-1.29913e-17,-3.80466e-25,7.71135e-17,2.20716e-12,-5.05665e-16,1.28898e-15,-2.17283e-18,-1.38946e-15,-2.94498e-18,-1.80127e-16,6.89979e-17,1.38122e-19,1.45478e-16,-6.05699e-24,-2.2897e-16,-1.05334e-17,-7.22898e-13,-5.67248e-14,9.4011e-17,-1.54035e-14,-3.63018e-24,8.03343e-13,-5.30699e-14,3.14779e-17,1.94062e-17,-2.61158e-16,1.35868e-20,-3.29982e-20,-7.09356e-15,2.618e-16,3.10862e-15,-6.02377e-19,-1.30696e-18,-2.48604e-15,-2.71738e-14,-8.57658e-15,-1.6687e-13,1.00758e-19,-2.50836e-12,1.19547e-19,2.10358e-14,4.27615e-15,9.47087e-18,-5.9202e-18,2.24634e-15,4.17411e-14,-1.24899e-20,-1.74303e-13,4.59608e-14,-1.02321e-19,4.14144e-16,2.40426e-14,3.66703e-15,-9.77263e-24,3.66014e-17,-2.44244e-24,3.229e-21,-2.60797e-20,1.85441e-13,1.45734e-19,-2.56036e-14,1.30472e-14,6.41249e-16,-2.56509e-13,-9.56595e-14,-1.11219e-22,8.21501e-19,-1.61301e-16,-7.12184e-14,2.28414e-14,-4.84084e-22,-2.69397e-17,-4.41572e-15,4.08602e-19,-2.21089e-22,9.57239e-14,8.50374e-17,-2.95555e-16,9.4404e-21,-2.8986e-14,-1.49541e-19,1.55441e-16,1.08834e-14,3.22021e-13,-1.32685e-14,6.6143e-18,-2.42117e-20,-2.31583e-19,-2.84111e-13,-1.97217e-13,3.92507e-17,1.89424e-16,1.30981e-17,-5.85837e-13,6.29368e-13,-7.08922e-13,1.6422e-14,2.02774e-13,8.10867e-13,5.79142e-12,6.33716e-21,-3.87916e-24,-3.40453e-14,1.4468e-15,-2.39038e-21,-1.73244e-14,8.9903e-13,6.95502e-15,-2.10017e-20,3.13159e-21,-9.40482e-13,-1.13076e-14,-7.06816e-15,1.17627e-17,-1.07579e-13,4.37365e-14,-8.78794e-14,-1.40374e-15,1.49961e-15,2.14342e-13,1.11301e-16,4.28285e-17,-1.52502e-25,2.23413e-13,-2.73022e-18,5.48085e-17,2.27107e-17,-1.95201e-20,-6.45913e-14,6.63751e-19,-1.20665e-15,3.52532e-25,8.94468e-12,8.87076e-17,1.26728e-21,-9.63209e-25,-1.25828e-16,-1.36549e-14,1.10831e-17,-3.37985e-15,-4.82806e-30,7.8002e-16,-5.79001e-15,5.38462e-20,7.74524e-15,3.28314e-15,-4.06259e-16,1.87437e-16,-1.24107e-12,-2.86716e-21,4.79928e-26,-1.10779e-14,1.07171e-18,-2.37803e-14,-7.81732e-22,8.95815e-17,-8.9861e-19,-6.59043e-23,-3.24679e-15,1.04647e-19,-2.73431e-15,1.10512e-15,-7.12317e-16,1.33589e-11,-6.25412e-18,-3.35499e-19,-4.97422e-17,4.19588e-17,4.839e-21,2.13849e-14,-1.47444e-16,-2.82748e-13,-2.51836e-18,-6.432e-12,6.99373e-13,-5.80293e-14,-7.95462e-14,-1.2647e-21,5.60972e-19,4.38888e-15,1.23375e-17,-4.662e-16,8.03248e-16,8.67055e-15,-3.20438e-15,-1.65888e-15,2.34292e-19,-2.43334e-14,-9.44596e-27,1.69359e-19,5.26503e-17,1.07651e-14,-3.59575e-14,3.07231e-14,6.36673e-19,-1.5592e-15,1.39384e-13,-2.67196e-18,-2.42268e-21,-6.25138e-21,1.43313e-13,-1.22846e-16,-2.52311e-17,-4.08483e-25,-3.47832e-16,-2.83313e-13,-2.08562e-14,2.79147e-15,-2.08393e-15,8.07621e-19,2.69627e-17,-8.29209e-17,1.43106e-13,8.42249e-19,4.06294e-14,3.62436e-17,-7.54617e-19,3.93149e-16,-9.6544e-17,1.40789e-13,2.12915e-16,4.08902e-19,3.12805e-15,1.4702e-27,1.31919e-14,1.59645e-21,4.16246e-17,-2.18489e-15,-1.09916e-14,-5.63475e-16,2.17545e-16,1.14844e-14,2.68605e-14,1.3889e-16,1.79671e-16,-6.17909e-16,-6.63603e-16,1.63406e-17,-2.94327e-15,2.09649e-19,-5.49624e-15,1.49344e-15,-2.59936e-16,9.15833e-15,-1.83914e-18,3.44777e-17,-1.06596e-24,5.1477e-15,-2.93557e-13,-9.44688e-17,1.32051e-15,6.2463e-17,-6.60169e-17,2.12551e-20,1.43263e-22,1.68779e-19,4.23686e-13,-2.8009e-19,-7.58355e-26,1.28936e-28,4.27389e-12,2.31485e-14,-2.3983e-14,2.7959e-15,-4.11717e-14,-7.81281e-16,-5.93839e-17,-2.65132e-19,5.52574e-17,-4.39762e-17,8.91243e-17,-1.51722e-14,2.80173e-15,4.59371e-17,-6.08335e-15,-1.95057e-22,1.76303e-15,-4.29994e-21,1.33051e-20,6.33175e-15,-1.2266e-14,-5.0368e-21,-2.77602e-26,1.30163e-24,1.35714e-20,-9.34029e-16,-3.34685e-19,-1.58969e-15,-5.39037e-15,-5.85516e-21,-2.56934e-19,7.81248e-16,3.09779e-18,1.41916e-17,-1.30917e-14,-8.20146e-17,-2.48346e-14,-3.97421e-13,3.17487e-20,-4.03989e-14,2.65301e-15,-7.9587e-20,4.0994e-11,1.81782e-16,-6.61889e-14,-3.51805e-15,2.57611e-17,-4.50849e-15,-1.39841e-20,-1.90305e-16,2.00566e-14,1.55261e-16,-5.89001e-18,2.46035e-14,1.29093e-22,1.80777e-12,-6.94019e-15,8.02861e-21,-1.003e-14,9.8063e-14,4.33662e-14,-8.53207e-19,8.96025e-17,-2.54722e-16,-4.1065e-19,1.65667e-16,1.26739e-16,2.36889e-14,2.94227e-12,9.22583e-17,3.69899e-14,-1.00382e-16,6.08132e-24,-4.20487e-26,-4.69697e-21,-4.59841e-17,-1.91694e-15,-1.92205e-17,9.53564e-14,1.9002e-19,1.58535e-14,2.25222e-26,-7.24825e-15,5.8021e-19,2.60976e-15,4.30633e-18,4.15305e-17,7.7001e-16,-1.19133e-13,4.00113e-16,6.14378e-17,3.49915e-26,-1.20953e-23,7.03378e-15,-3.85752e-17,4.3441e-20'.split(','))

512

In [50]:
len('-1.08668e-23,-8.38863e-17,-1.68725e-14,6.09481e-12,-2.61027e-17,5.49815e-15,-9.11604e-24,-5.71803e-20,6.2054e-15,-1.06939e-15,-1.56427e-14,-1.61474e-15,-4.26515e-14,-3.18455e-20,-9.61656e-18,3.81575e-17,-3.15913e-16,1.23304e-16,5.51392e-14,2.6534e-14,2.11093e-14,-2.86068e-14,-2.06027e-16,-2.17444e-15,-1.97737e-21,1.12713e-17,-4.45558e-17,2.20851e-19,-1.58396e-22,2.82563e-16,-1.57765e-16,-1.76483e-15,5.13024e-17,-3.95336e-18,-3.14162e-29,1.25582e-14,-5.17659e-17,-1.77439e-18,-1.76176e-16,-3.33907e-24,-9.90975e-15,-1.08691e-14,2.49743e-15,8.17084e-14,4.94833e-17,1.10987e-13,-2.60064e-18,1.00588e-16,-1.33177e-17,8.24316e-24,2.60431e-19,-1.87382e-13,1.81284e-23,4.59156e-20,5.7229e-21,1.73087e-16,-1.9062e-16,-4.74507e-19,-7.74557e-16,4.03376e-25,2.90085e-17,-5.42029e-13,-8.60066e-17,4.79003e-14,7.8699e-17,1.19824e-14,2.64726e-33,-3.04495e-14,6.27227e-22,3.71611e-18,5.56041e-14,-2.07708e-13,-5.37779e-15,1.51282e-22,-3.13351e-17,-5.07865e-18,5.28074e-27,-6.1907e-14,3.55174e-14,5.04606e-15,-8.10712e-13,-4.90654e-17,-4.42282e-14,6.03208e-17,-5.86198e-16,-4.91869e-13,-8.47756e-12,5.05601e-15,8.32781e-17,-1.61176e-14,6.11114e-17,4.19634e-17,7.29077e-18,5.26367e-17,3.56421e-16,3.77776e-22,1.48139e-16,5.29807e-14,-2.6544e-21,-3.4521e-15,5.16427e-16,1.63959e-23,-2.98885e-20,-2.55533e-16,-5.1067e-14,-5.04867e-16,1.5417e-22,-1.79615e-12,-3.52633e-17,-5.46237e-16,6.73562e-13,7.17307e-17,5.86539e-16,3.75626e-13,1.56769e-16,8.60966e-16,-7.70546e-14,-3.1134e-18,1.5134e-19,-6.79793e-26,-2.02269e-20,-2.33876e-22,-3.21231e-19,-2.69463e-15,-1.52162e-14,1.71232e-14,-3.16799e-28,-9.37616e-20,-1.59132e-16,-1.21923e-18,2.04483e-19,1.89772e-16,2.17358e-16,-1.2823e-19,-3.28201e-12,-2.16663e-14,-1.36232e-14,8.42443e-21,-8.33071e-17,1.50272e-16,-1.34193e-14,-6.97211e-13,-6.02458e-21,-1.76116e-19,1.29205e-18,-1.68466e-27,1.82134e-14,-2.65382e-16,-4.32351e-10,-1.57436e-22,-4.00512e-15,6.24643e-22,-5.1352e-16,2.82359e-16,1.47334e-17,1.15443e-15,-1.29913e-17,-3.80466e-25,7.71135e-17,2.20716e-12,-5.05665e-16,1.28898e-15,-2.17283e-18,-1.38946e-15,-2.94498e-18,-1.80127e-16,6.89979e-17,1.38122e-19,1.45478e-16,-6.05699e-24,-2.2897e-16,-1.05334e-17,-7.22898e-13,-5.67248e-14,9.4011e-17,-1.54035e-14,-3.63018e-24,8.03343e-13,-5.30699e-14,3.14779e-17,1.94062e-17,-2.61158e-16,1.35868e-20,-3.29982e-20,-7.09356e-15,2.618e-16,3.10862e-15,-6.02377e-19,-1.30696e-18,-2.48604e-15,-2.71738e-14,-8.57658e-15,-1.6687e-13,1.00758e-19,-2.50836e-12,1.19547e-19,2.10358e-14,4.27615e-15,9.47087e-18,-5.9202e-18,2.24634e-15,4.17411e-14,-1.24899e-20,-1.74303e-13,4.59608e-14,-1.02321e-19,4.14144e-16,2.40426e-14,3.66703e-15,-9.77263e-24,3.66014e-17,-2.44244e-24,3.229e-21,-2.60797e-20,1.85441e-13,1.45734e-19,-2.56036e-14,1.30472e-14,6.41249e-16,-2.56509e-13,-9.56595e-14,-1.11219e-22,8.21501e-19,-1.61301e-16,-7.12184e-14,2.28414e-14,-4.84084e-22,-2.69397e-17,-4.41572e-15,4.08602e-19,-2.21089e-22,9.57239e-14,8.50374e-17,-2.95555e-16,9.4404e-21,-2.8986e-14,-1.49541e-19,1.55441e-16,1.08834e-14,3.22021e-13,-1.32685e-14,6.6143e-18,-2.42117e-20,-2.31583e-19,-2.84111e-13,-1.97217e-13,3.92507e-17,1.89424e-16,1.30981e-17,-5.85837e-13,6.29368e-13,-7.08922e-13,1.6422e-14,2.02774e-13,8.10867e-13,5.79142e-12,6.33716e-21,-3.87916e-24,-3.40453e-14,1.4468e-15,-2.39038e-21,-1.73244e-14,8.9903e-13,6.95502e-15,-2.10017e-20,3.13159e-21,-9.40482e-13,-1.13076e-14,-7.06816e-15,1.17627e-17,-1.07579e-13,4.37365e-14,-8.78794e-14,-1.40374e-15,1.49961e-15,2.14342e-13,1.11301e-16,4.28285e-17,-1.52502e-25,2.23413e-13,-2.73022e-18,5.48085e-17,2.27107e-17,-1.95201e-20,-6.45913e-14,6.63751e-19,-1.20665e-15,3.52532e-25,8.94468e-12,8.87076e-17,1.26728e-21,-9.63209e-25,-1.25828e-16,-1.36549e-14,1.10831e-17,-3.37985e-15,-4.82806e-30,7.8002e-16,-5.79001e-15,5.38462e-20,7.74524e-15,3.28314e-15,-4.06259e-16,1.87437e-16,-1.24107e-12,-2.86716e-21,4.79928e-26,-1.10779e-14,1.07171e-18,-2.37803e-14,-7.81732e-22,8.95815e-17,-8.9861e-19,-6.59043e-23,-3.24679e-15,1.04647e-19,-2.73431e-15,1.10512e-15,-7.12317e-16,1.33589e-11,-6.25412e-18,-3.35499e-19,-4.97422e-17,4.19588e-17,4.839e-21,2.13849e-14,-1.47444e-16,-2.82748e-13,-2.51836e-18,-6.432e-12,6.99373e-13,-5.80293e-14,-7.95462e-14,-1.2647e-21,5.60972e-19,4.38888e-15,1.23375e-17,-4.662e-16,8.03248e-16,8.67055e-15,-3.20438e-15,-1.65888e-15,2.34292e-19,-2.43334e-14,-9.44596e-27,1.69359e-19,5.26503e-17,1.07651e-14,-3.59575e-14,3.07231e-14,6.36673e-19,-1.5592e-15,1.39384e-13,-2.67196e-18,-2.42268e-21,-6.25138e-21,1.43313e-13,-1.22846e-16,-2.52311e-17,-4.08483e-25,-3.47832e-16,-2.83313e-13,-2.08562e-14,2.79147e-15,-2.08393e-15,8.07621e-19,2.69627e-17,-8.29209e-17,1.43106e-13,8.42249e-19,4.06294e-14,3.62436e-17,-7.54617e-19,3.93149e-16,-9.6544e-17,1.40789e-13,2.12915e-16,4.08902e-19,3.12805e-15,1.4702e-27,1.31919e-14,1.59645e-21,4.16246e-17,-2.18489e-15,-1.09916e-14,-5.63475e-16,2.17545e-16,1.14844e-14,2.68605e-14,1.3889e-16,1.79671e-16,-6.17909e-16,-6.63603e-16,1.63406e-17,-2.94327e-15,2.09649e-19,-5.49624e-15,1.49344e-15,-2.59936e-16,9.15833e-15,-1.83914e-18,3.44777e-17,-1.06596e-24,5.1477e-15,-2.93557e-13,-9.44688e-17,1.32051e-15,6.2463e-17,-6.60169e-17,2.12551e-20,1.43263e-22,1.68779e-19,4.23686e-13,-2.8009e-19,-7.58355e-26,1.28936e-28,4.27389e-12,2.31485e-14,-2.3983e-14,2.7959e-15,-4.11717e-14,-7.81281e-16,-5.93839e-17,-2.65132e-19,5.52574e-17,-4.39762e-17,8.91243e-17,-1.51722e-14,2.80173e-15,4.59371e-17,-6.08335e-15,-1.95057e-22,1.76303e-15,-4.29994e-21,1.33051e-20,6.33175e-15,-1.2266e-14,-5.0368e-21,-2.77602e-26,1.30163e-24,1.35714e-20,-9.34029e-16,-3.34685e-19,-1.58969e-15,-5.39037e-15,-5.85516e-21,-2.56934e-19,7.81248e-16,3.09779e-18,1.41916e-17,-1.30917e-14,-8.20146e-17,-2.48346e-14,-3.97421e-13,3.17487e-20,-4.03989e-14,2.65301e-15,-7.9587e-20,4.0994e-11,1.81782e-16,-6.61889e-14,-3.51805e-15,2.57611e-17,-4.50849e-15,-1.39841e-20,-1.90305e-16,2.00566e-14,1.55261e-16,-5.89001e-18,2.46035e-14,1.29093e-22,1.80777e-12,-6.94019e-15,8.02861e-21,-1.003e-14,9.8063e-14,4.33662e-14,-8.53207e-19,8.96025e-17,-2.54722e-16,-4.1065e-19,1.65667e-16,1.26739e-16,2.36889e-14,2.94227e-12,9.22583e-17,3.69899e-14,-1.00382e-16,6.08132e-24,-4.20487e-26,-4.69697e-21,-4.59841e-17,-1.91694e-15,-1.92205e-17,9.53564e-14,1.9002e-19,1.58535e-14,2.25222e-26,-7.24825e-15,5.8021e-19,2.60976e-15,4.30633e-18,4.15305e-17,7.7001e-16,-1.19133e-13,4.00113e-16,6.14378e-17,3.49915e-26,-1.20953e-23,7.03378e-15,-3.85752e-17,4.3441e-20')

6347

In [14]:
number_dimensions_output

NameError: name 'number_dimensions_output' is not defined

In [24]:
vector_table.columns

['TD_ID', 'TD_FILENAME', 'txt', 'sentence_embedding']

In [25]:
print(tdml.execute_sql(f"SHOW TABLE {database}.{vector_store_table}").fetchall()[0][0].replace('\r','\n'))

CREATE MULTISET TABLE dm250067.denis_pdfs_embeddings ,FALLBACK ,
     NO BEFORE JOURNAL,
     NO AFTER JOURNAL,
     CHECKSUM = DEFAULT,
     DEFAULT MERGEBLOCKRATIO,
     MAP = TD_MAP2
     (
      TD_ID INTEGER,
      TD_FILENAME VARCHAR(1024) CHARACTER SET LATIN NOT CASESPECIFIC,
      txt VARCHAR(32000) CHARACTER SET LATIN NOT CASESPECIFIC,
      sentence_embedding SYSUDTLIB.Vector)
PRIMARY INDEX ( TD_ID );


### 3.3 - Vector Normalization: TD_VECTORNORMALIZE

In [46]:
vector_column            = 'sentence_embedding'
idcolumns                = ['TD_ID']
embedding_dimension      = int(number_dimensions_output/2)
accumulate_columns       = ['TD_FILENAME','txt']

In [47]:
query_normalization = f"""
SELECT *
FROM TD_Vectornormalize(
    ON {database}.{vector_store_table} AS InputTable
    USING
        IDColumns( {','.join(["'"+x+"'" for x in idcolumns])})
        TargetColumns('{vector_column}')
        Accumulate({','.join(["'"+x+"'" for x in accumulate_columns])})
        Approach('UNITVECTOR')
        EmbeddingSize({embedding_dimension})    
) as DT
"""

print(query_normalization)


SELECT *
FROM TD_Vectornormalize(
    ON dm250067.denis_pdfs_embeddings AS InputTable
    USING
        IDColumns( 'TD_ID')
        TargetColumns('sentence_embedding')
        Accumulate('TD_FILENAME','txt')
        Approach('UNITVECTOR')
        EmbeddingSize(512)    
) as DT



In [48]:
tdml.DataFrame.from_query(query_normalization)

select top 10 "TD_ID" AS "TD_ID", CAST(sentence_embedding AS VARCHAR(1024)) AS sentence_embedding, "TD_FILENAME" AS "TD_FILENAME", txt AS txt FROM "DM250067"."ml___frmqry_v_1745315927220934";


TD_ID,sentence_embedding,TD_FILENAME,txt
101,"4.4855e-09,5.60369e-16,-0.000214027,0.162462,-6.98586e-06,6.4996e-05,6.9795e-09,8.48514e-10,7.33777e-06,-1.39426e-05,4.06199e-08,-1.37146e-05,-4.12851e-07,-0.00026784,-2.78344e-11,-1.56669e-09,-0.00046857,-4.25468e-10,3.26161e-07,1.03454e-07,0.00342517,-6.08289e-07,8.54585e-06,-5.77784e-07,9.92904e-07,1.41762e-10,-6.65264e-06,3.89118e-05,3.25142e-07,-3.24409e-06,-3.89559e-06,-4.76092e-07,0.000109174,-1.03058e-06,1.09736e-08,3.70717e-06,-0.000302008,-6.32088e-07,-0.00078337,-0.000156958,-3.59168e-08,-0.000394303,0.00433779,0.00429595,8.54497e-05,0.00160144,-2.13004e-13,1.62114e-09,-6.4156e-12,7.43549e-16,-2.19432e-05,-0.000532877,3.42049e-12,1.623e-06,1.94473e-08,0.0858352,-2.03818e-06,-4.94366e-09,-1.12095e-05,8.823e-05,9.67827e-12,-0.0443551,1.46408e-09,0.000168999,6.95463e-06,3.54662e-09,2.22138e-12,-0.0077097,-4.56084e-05,6.95915e-06,1.72654e-06,-0.000392796,-0.0176501,6.9347e-07,6.73684e-09,-3.05156e-06,1.19563e-07,-6.54687e-06,0.00707374,-1.28358e-12,-0.000576795,-3.04393e-10,-4.2864e-08,6.18163e-05,-0.0",2404.16130v1.pdf,"ao, Z., Gong, Y ., Shen, Y ., Huang, M., Duan, N., and Chen, W. (2023). Enhancing retrieval- augmented large language models with iterative retrieval-generation synergy. arXiv preprint arXiv:2305.15294 . Su, D., Xu, Y ., Yu, T., Siddique, F. B., Barezi, E. J., and Fung, P. (2020). Caire-covid: A ques- tion answering and query-focused multi-document summarization system for covid-19 scholarly information management. arXiv preprint arXiv:2005.03975 . Tang, Y . and Yang, Y . (2024). MultiHop-RAG: B"
122,"2.97173e-11,-0.000967666,-2.01463e-05,0.119166,-1.74524e-07,1.86819e-14,1.71163e-09,-3.96588e-08,0.00122049,-1.03554e-05,-1.03277e-05,-3.25471e-05,-2.71898e-08,-3.63605e-07,-3.6574e-06,-4.23177e-11,-0.000339323,1.36463e-05,3.82636e-06,7.62226e-11,-2.70648e-13,-0.000498216,3.70805e-05,-2.64551e-07,9.07805e-07,2.83382e-07,-0.000886317,5.76342e-14,1.20484e-23,2.39448e-08,-6.6387e-06,-1.19778e-07,2.03548e-06,4.53685e-11,1.34013e-05,2.00906e-08,-3.00553e-07,-9.5847e-07,-0.000764897,-1.16692e-26,-9.51592e-07,-1.90666e-05,8.36501e-05,1.43161e-06,5.3497e-05,0.000470534,-3.60888e-08,8.00992e-07,1.53318e-11,2.5905e-08,-8.19251e-07,-0.000972887,1.00902e-16,0.00430252,2.08129e-08,4.24543e-07,-6.49126e-07,2.28493e-09,-3.77829e-10,1.42621e-05,-2.45914e-08,-0.00026219,-1.06934e-07,3.64997e-05,3.5727e-07,1.36151e-05,2.81803e-09,-0.00582549,-7.13957e-07,-6.20435e-07,0.00204534,-0.00204077,-0.0508751,-7.18928e-16,-2.53424e-08,-3.40408e-09,8.42164e-07,-7.15028e-13,0.000295378,7.23232e-10,-0.00041153,-2.57391e-07,-2.89596e-05,6.",2404.03622v3.pdf,"tep 1 (Move down): After Step 2 (Move right): After Step 3 (Move up): (b) Route planning task is sensitive to prompts. After deleting the word ""reasoning"" from the V oT prompt, final answer and state is derived without conditioning on state visualization in many cases. The left example provides a correct solution with imperfect intermediate visualization, while the right example provides a wrong solution. Figure 8: Example cases of visual state tracking behaviors in different tasks. We provide s"
19,"2.08273e-07,-7.74028e-07,-2.44999e-11,0.0279226,-1.50269e-06,8.56518e-06,3.6812e-10,-3.30516e-08,1.30723e-09,-1.50838e-08,1.41569e-09,-2.68304e-06,-4.26207e-09,-6.10406e-07,-6.11319e-07,3.62221e-19,-8.56476e-05,4.52358e-05,3.27561e-08,1.4028e-10,0.000986129,-9.02584e-07,1.61842e-06,-6.31551e-05,8.83399e-12,2.33833e-10,-3.68501e-07,6.72709e-05,1.4192e-09,7.93434e-08,-9.48485e-09,-1.00165e-07,1.58873e-07,-4.66597e-10,4.15434e-07,1.90413e-05,-1.70212e-06,-4.11858e-07,-4.19411e-08,-3.37829e-09,-4.75805e-09,-1.64054e-06,0.000481595,5.87177e-05,5.02802e-05,0.00162682,-1.75901e-08,-3.57514e-09,5.84978e-06,3.05947e-32,-1.75312e-09,-2.65951e-05,0.000284584,2.63904e-09,2.79611e-12,0.000264744,-5.60187e-07,-7.39268e-16,-5.28165e-08,1.58956e-06,1.93901e-17,-0.000384332,5.93178e-07,1.94811e-05,-4.91355e-13,4.44251e-16,2.1448e-15,-4.56777e-05,-6.69989e-